In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
normal = pd.read_excel('normaldata.xlsx', header = None).to_numpy()
random = pd.read_excel('randomdata.xlsx', header = None).to_numpy()

normalt = pd.read_excel('normalt.xlsx', header = None).to_numpy()
randomt = pd.read_excel('randomt.xlsx', header = None).to_numpy()

normalkapa = pd.read_excel('normalkapa.xlsx', header = None).to_numpy()
randomkapa = pd.read_excel('randomkapa.xlsx', header = None).to_numpy()

normalkapaori = pd.read_excel('normalkapaori.xlsx', header = None).to_numpy()
randomkapaori = pd.read_excel('randomkapaori.xlsx', header = None).to_numpy()

normaltau = pd.read_excel('normaltau.xlsx', header = None).to_numpy()
randomtau = pd.read_excel('randomtau.xlsx', header = None).to_numpy()

normaltauori = pd.read_excel('normaltauori.xlsx', header = None).to_numpy()
randomtauori = pd.read_excel('randomtauori.xlsx', header = None).to_numpy()

normal = torch.from_numpy(normal).float().to(device)
random = torch.from_numpy(random).float().to(device)

normalt = torch.from_numpy(normalt).float().to(device)
randomt = torch.from_numpy(randomt).float().to(device)

normalkapa = torch.from_numpy(normalkapa).float().to(device)
randomkapa = torch.from_numpy(randomkapa).float().to(device)

normalkapaori = torch.from_numpy(normalkapaori).float().to(device)
randomkapaori = torch.from_numpy(randomkapaori).float().to(device)

normaltau = torch.from_numpy(normaltau).float().to(device)
randomtau = torch.from_numpy(randomtau).float().to(device)

normaltauori = torch.from_numpy(normaltauori).float().to(device)
randomtauori = torch.from_numpy(randomtauori).float().to(device)

In [5]:
# get lengths of all tubes
normallength1 = (normalt[:, 9] + normalt[:, 19] + normalt[:, 29] + normalt[:, 39] + normalt[:, 49]) * normal[:, 4]
R1, R2, R3, R4 = normal[:, 6], normal[:, 6], normal[:, 7], normal[:, 8]
P1, P2, P3, P4 = normal[:, 9], normal[:, 10], normal[:, 10], normal[:, 11]
length2 = torch.sqrt((2 * torch.pi * R1) ** 2 + P1 ** 2) / (2 * torch.pi) * (normal[:, 17] / 180 * torch.pi)
length3 = torch.sqrt((2 * torch.pi * R2) ** 2 + P2 ** 2) / (2 * torch.pi) * (normal[:, 18] / 180 * torch.pi)
length4 = torch.sqrt((2 * torch.pi * R3) ** 2 + P3 ** 2) / (2 * torch.pi) * (normal[:, 19] / 180 * torch.pi)
length5 = torch.sqrt((2 * torch.pi * R4) ** 2 + P4 ** 2) / (2 * torch.pi) * (normal[:, 20] / 180 * torch.pi)
normallength = normallength1 + length2 + length3 + length4 + length5

randomlength = (randomt[:, 109] + randomt[:, 119]) * random[:, 4]

normallength = normallength.unsqueeze(1)
randomlength = randomlength.unsqueeze(1)

normallength_normalize = normallength / 1000
randomlength_normalize = randomlength / 1000

In [6]:
index1 = [0, 1, 2, 3, 4, 5, 21]
index2 = [0, 1, 2, 3, 4, 5, 25]

feature1 = normal[:, index1]
kapaori1 = normalkapaori * 100
kapa1 = normalkapa * 100
tauori1 = normaltauori * 200
tau1 = normaltau * 200

feature2 = random[:, index2]
kapaori2 = randomkapaori * 100
kapa2 = randomkapa * 100
tauori2 = randomtauori * 200
tau2 = randomtau * 200

ranges = [(20, 35), (0.075, 0.1), (1.3, 1.6), (1.05, 1.35), (40, 100), (0.05, 0.25), (0, 0.001)]
feature_normalize1 = torch.zeros_like(feature1)
feature_normalize2 = torch.zeros_like(feature2)
for i in range(7):
    minvalue, maxvalue = ranges[i]
    feature_normalize1[:, i] = (feature1[:, i] - minvalue) / (maxvalue - minvalue)
    feature_normalize2[:, i] = (feature2[:, i] - minvalue) / (maxvalue - minvalue)

feature_normalize1 = torch.hstack((feature_normalize1, normallength_normalize))
feature_normalize2 = torch.hstack((feature_normalize2, randomlength_normalize))

t1 = torch.linspace(0, 1, 1200).reshape(1200, 1)
t_s = t1.to(device)

frac_num1 = int(kapaori1.shape[0] * 0.7)
frac_num2 = int(kapaori2.shape[0] * 0.7)
# frac_num2 = int(kapaori2.shape[0] * 0.8)
train_feature1, test_feature1 = feature_normalize1[:frac_num1], feature_normalize1[frac_num1:]
train_feature2, test_feature2 = feature_normalize2[:frac_num2], feature_normalize2[frac_num2:]

train_kapaori1, test_kapaori1 = kapaori1[:frac_num1], kapaori1[frac_num1:]
train_kapaori2, test_kapaori2 = kapaori2[:frac_num2], kapaori2[frac_num2:]

train_kapa1, test_kapa1 = kapa1[:frac_num1], kapa1[frac_num1:]
train_kapa2, test_kapa2 = kapa2[:frac_num2], kapa2[frac_num2:]

train_tauori1, test_tauori1 = tauori1[:frac_num1], tauori1[frac_num1:]
train_tauori2, test_tauori2 = tauori2[:frac_num2], tauori2[frac_num2:]

train_tau1, test_tau1 = tau1[:frac_num1], tau1[frac_num1:]
train_tau2, test_tau2 = tau2[:frac_num2], tau2[frac_num2:]

train_feature = torch.vstack((train_feature1, train_feature2))
test_feature = torch.vstack((test_feature1, test_feature2))
train_kapaori = torch.vstack((train_kapaori1, train_kapaori2))
test_kapaori = torch.vstack((test_kapaori1, test_kapaori2))
train_kapa = torch.vstack((train_kapa1, train_kapa2))
test_kapa = torch.vstack((test_kapa1, test_kapa2))
train_tauori = torch.vstack((train_tauori1, train_tauori2))
test_tauori = torch.vstack((test_tauori1, test_tauori2))
train_tau = torch.vstack((train_tau1, train_tau2))
test_tau = torch.vstack((test_tau1, test_tau2))

In [7]:
mean = np.mean(train_kapaori.cpu().numpy(), axis=0)
centered_data = train_kapaori.cpu().numpy() - mean[None, :]
covariance_matrix = np.cov(centered_data, rowvar=False)
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
basis = eigenvectors

basis = torch.from_numpy(basis).float().to(device)

In [8]:
x_train = (train_kapaori, train_tauori, train_feature, t_s)
y_train = train_kapa
x_test = (test_kapaori, test_tauori, test_feature, t_s)
y_test = test_kapa

In [9]:
class CustomDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __len__(self):
        return len(self.data[0])
    
    def __getitem__(self, idx):
        x1 = self.data[0]
        x2 = self.data[1]
        x3 = self.data[2]
        x4 = self.data[3]
        x11 = x1[idx]
        x22 = x2[idx]
        x33 = x3[idx]
        x44 = x4
        labels = self.label[idx]
        x = (x11, x22, x33, x44)
        return x, labels

train_loader = DataLoader(CustomDataset(x_train, y_train), batch_size = 64, shuffle = True)
test_loader = DataLoader(CustomDataset(x_test, y_test), batch_size = 64, shuffle = False)

In [10]:
class FNN(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.dense = nn.ModuleList()
        for i in range(1, len(layer_sizes) - 1):
            self.dense.append(
                nn.Linear(in_features = layer_sizes[i - 1], out_features = layer_sizes[i])
            )
            # self.dense.append(nn.BatchNorm1d(layer_sizes[i]))
            self.dense.append(nn.Tanh())
            # self.dense.append(nn.Dropout(p = 0.2))
        self.dense.append(
            nn.Linear(in_features = layer_sizes[-2], out_features = layer_sizes[-1])
        )
        # self.dense.append(nn.Tanh())

    def forward(self, inputs):
        y = inputs
        for layer in self.dense:
            y = layer(y)
        return y

In [11]:
class MIONet(nn.Module):
    def __init__(self, branch1, branch2, branch3, branch4, branch5, branch6, basis, layers1, layers2, layers3, layers4):
        super().__init__()
        self.branch1 = branch1
        self.branch2 = branch2
        self.branch3 = branch3
        self.branch4 = branch4
        self.branch5 = branch5
        self.branch6 = branch6
        self.basis = basis
        self.linears1 = torch.nn.ModuleList([torch.nn.Linear(layers1[i], layers1[i+1]) for i in range(len(layers1)-1)])
        self.linears2 = torch.nn.ModuleList([torch.nn.Linear(layers2[i], layers2[i+1]) for i in range(len(layers2)-1)]) 
        self.linears3 = torch.nn.ModuleList([torch.nn.Linear(layers3[i], layers3[i+1]) for i in range(len(layers3)-1)]) 
        self.linears4 = torch.nn.ModuleList([torch.nn.Linear(layers4[i], layers4[i+1]) for i in range(len(layers4)-1)]) 
        self.layers1 = layers1
        self.layers2 = layers2
        self.layers3 = layers3
        self.layers4 = layers4
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction = 'mean')
        self.w = nn.parameter.Parameter(torch.tensor(1.0))

    def forward0(self, inputs):
        x_func1, x_func2, x_func3, x_loc = inputs
        if len(x_loc.shape) == 3:
            x_loc = x_loc[0, :]
        else:              
            x_loc = x_loc
        x_loc = torch.hstack((x_loc, torch.sin(x_loc * 2 * torch.pi * self.w), torch.cos(x_loc * 2 * torch.pi * self.w), torch.sin(2 * x_loc * 2 * torch.pi * self.w), torch.cos(2 * x_loc * 2 * torch.pi * self.w), torch.sin(3 * x_loc * 2 * torch.pi * self.w), torch.cos(3 * x_loc * 2 * torch.pi * self.w), torch.sin(4 * x_loc * 2 * torch.pi * self.w), torch.cos(4 * x_loc * 2 * torch.pi * self.w), torch.sin(5 * x_loc * 2 * torch.pi * self.w), torch.cos(5 * x_loc * 2 * torch.pi * self.w)))
        H1 = self.branch1(x_func1)
        H2 = self.branch2(x_func2)
        H3 = self.branch3(x_func3)
        H4 = self.branch4(x_loc)
        H4 = self.branch5(torch.transpose(H4, 0, 1)) # [1, 100]

        return H1, H2, H3, H4
    
    def forward1(self, inputs):
        H1, H2, H3, H4 = self.forward0(inputs)
        h = inputs[0]

        for i in range(len(self.layers1) - 2):
            z = self.activation(self.linears1[i](h))
            # h = ((1 - z) * H1 + z * H2) * H3
            h1 = (1 - z) * H1 + z * H4
            h2 = (1 - z) * H2 + z * H4
            h3 = (1 - z) * H3 + z * H4
            h = h1 + h2 + h3
        a = self.linears1[-1](h)
        return a
    
    def forward2(self, inputs):
        H1, H2, H3, H4 = self.forward0(inputs)
        h = inputs[1]
  
        for i in range(len(self.layers2) - 2):
            z = self.activation(self.linears2[i](h))
            # h = ((1 - z) * H1 + z * H2) * H3
            h1 = (1 - z) * H1 + z * H4
            h2 = (1 - z) * H2 + z * H4
            h3 = (1 - z) * H3 + z * H4
            h = h1 + h2 + h3
        a = self.linears2[-1](h)
        return a
    
    def forward3(self, inputs):
        H1, H2, H3, H4 = self.forward0(inputs)
        h = inputs[2]
  
        for i in range(len(self.layers3) - 2):
            z = self.activation(self.linears3[i](h))
            # h = ((1 - z) * H1 + z * H2) * H3
            h1 = (1 - z) * H1 + z * H4
            h2 = (1 - z) * H2 + z * H4
            h3 = (1 - z) * H3 + z * H4
            h = h1 + h2 + h3
        a = self.linears3[-1](h)
        return a

    def forward4(self, inputs):
        x_loc = inputs[3]
        if len(x_loc.shape) == 3:
            x_loc = x_loc[0, :]
        else:
            x_loc = x_loc
        x_loc = torch.hstack((x_loc, torch.sin(x_loc * 2 * torch.pi * self.w), torch.cos(x_loc * 2 * torch.pi * self.w), torch.sin(2 * x_loc * 2 * torch.pi * self.w), torch.cos(2 * x_loc * 2 * torch.pi * self.w), torch.sin(3 * x_loc * 2 * torch.pi * self.w), torch.cos(3 * x_loc * 2 * torch.pi * self.w), torch.sin(4 * x_loc * 2 * torch.pi * self.w), torch.cos(4 * x_loc * 2 * torch.pi * self.w), torch.sin(5 * x_loc * 2 * torch.pi * self.w), torch.cos(5 * x_loc * 2 * torch.pi * self.w))) # [50, 3]
        h = x_loc

        # basis2 = compute_eigenvectors(inputs[0])

        for i in range(len(self.layers4) - 2):
            z = self.activation(self.linears4[i](h)) # [50, 50]
            h = z
        a = self.linears4[-1](h)
        return a

    def forward(self, inputs):
        x_func3 = inputs[2]
        x_loc = inputs[3]
        if len(x_loc.shape) == 3:
            x_loc = x_loc[0, :]
        else:
            x_loc = x_loc
        
        y_func1 = self.forward1(inputs)
        y_func2 = self.forward2(inputs)
        y_func3 = self.forward3(inputs)
        y_loc = self.forward4(inputs)
        x_merger = torch.mul(y_func1, y_func3) + torch.mul(y_func2, y_func3)
        # x_merger = (torch.mul(y_func1, y_func3) + torch.mul(y_func2, y_func3)) / 2
        # x_merger = torch.mul(y_func2, y_func3)
        y_func = x_merger
        y_loc = torch.cat((self.basis, y_loc), axis = 1)
        b = self.branch6(x_func3)
        # y_loc = y_loc
        y1 = torch.einsum("ip, jp -> ij", y_func, y_loc)

        y = y1 + b

        return y
    

    def loss(self, inputs, outputs):
        out = self.forward(inputs)
        loss1 = self.loss_function(out, outputs)
        loss2 = torch.mean(out[:, 0] ** 2) + torch.mean(out[:, -1] ** 2)

        diff1 = (out[:, :-2] - 2 * out[:, 1:-1] + out[:, 2:]) / 10
        smooth_loss = torch.mean(diff1 ** 2)

        return loss1 + loss2 + smooth_loss
    

In [12]:
layer_sizes1 = [1200, 2400, 2400]
layer_sizes2 = [1200, 2400, 2400]
layer_sizes3 = [8, 2400, 2400]
# layer_sizes3 = [8, 1600, 2400]
layer_sizes4 = [11, 1]
layer_sizes5 = [1200, 2400, 2400]
layer_sizes6 = [8, 2400, 2400, 1200]

branch1 = FNN(layer_sizes1).to(device)
branch2 = FNN(layer_sizes2).to(device)
branch3 = FNN(layer_sizes3).to(device)
branch4 = FNN(layer_sizes4).to(device)
branch5 = FNN(layer_sizes5).to(device)
branch6 = FNN(layer_sizes6).to(device)

layers1 = [1200, 2400, 2400, 2400, 2400, 2400]
layers2 = [1200, 2400, 2400, 2400, 2400, 2400]
layers3 = [8, 2400, 2400, 2400, 2400, 2400]
# layers3 = [3, 50, 50, 50, 50, 50]
layers4 = [11, 1200, 1200, 1200, 1200, 1200]

lr = 1e-5
model = MIONet(branch1, branch2, branch3, branch4, branch5, branch6, basis, layers1, layers2, layers3, layers4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = lr, weight_decay = 0.0008)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1000, gamma = 0.5)

In [13]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        torch.nn.init.xavier_normal_(m.weight.data)
model.apply(initialize_weights)

MIONet(
  (branch1): FNN(
    (dense): ModuleList(
      (0): Linear(in_features=1200, out_features=2400, bias=True)
      (1): Tanh()
      (2): Linear(in_features=2400, out_features=2400, bias=True)
    )
  )
  (branch2): FNN(
    (dense): ModuleList(
      (0): Linear(in_features=1200, out_features=2400, bias=True)
      (1): Tanh()
      (2): Linear(in_features=2400, out_features=2400, bias=True)
    )
  )
  (branch3): FNN(
    (dense): ModuleList(
      (0): Linear(in_features=8, out_features=2400, bias=True)
      (1): Tanh()
      (2): Linear(in_features=2400, out_features=2400, bias=True)
    )
  )
  (branch4): FNN(
    (dense): ModuleList(
      (0): Linear(in_features=11, out_features=1, bias=True)
    )
  )
  (branch5): FNN(
    (dense): ModuleList(
      (0): Linear(in_features=1200, out_features=2400, bias=True)
      (1): Tanh()
      (2): Linear(in_features=2400, out_features=2400, bias=True)
    )
  )
  (branch6): FNN(
    (dense): ModuleList(
      (0): Linear(in_featu

In [14]:
def mean_absolute_error(y_true, y_pred):
    mae = torch.mean(torch.abs(y_true - y_pred))
    return mae

def standard_deviation_error(y_true, y_pred):
    sdm = torch.std(y_true - y_pred)
    return sdm

In [15]:
steps = 5000
min_loss = 0.004
loss_list = []
test_loss_list = []
for i in range(steps):
    model.train()
    for inputs, outputs in train_loader:
        optimizer.zero_grad()
        loss = model.loss(inputs, outputs)
        loss.backward(retain_graph = True)
        optimizer.step()
    loss_list.append(loss.item())
    scheduler.step()

    model.eval()
    total_test_loss = 0
    total_test_samples = 0
    with torch.no_grad():
        for inputs, outputs in test_loader:
            test_loss = model.loss(inputs, outputs)
            total_test_loss += test_loss.item() * len(inputs)
            total_test_samples += len(inputs)

        average_test_loss = total_test_loss / total_test_samples
        test_loss_list.append(average_test_loss)

    if (i + 1) % 1000 == 0:
        print('+++++++++++++', i + 1, '+++++++++++++')
        print('loss is: ', loss.detach().cpu().numpy())
        print('test_loss is: ', average_test_loss)

    if loss < min_loss and average_test_loss < 0.009:
        min_loss = loss
        torch.save(model.state_dict(), 'model1.pth')
        print('min_loss is: ', min_loss.detach().cpu().numpy(), 'loss is', average_test_loss)

+++++++++++++ 1000 +++++++++++++
loss is:  0.014393759
test_loss is:  0.014602222169438997
min_loss is:  0.0013100291 loss is 0.008923216878126064
min_loss is:  0.0012663002 loss is 0.008907788277914127
min_loss is:  0.0011855827 loss is 0.008627714744458595
min_loss is:  0.0010729427 loss is 0.008041825300703445
min_loss is:  0.0010453345 loss is 0.008108576914916435
min_loss is:  0.001013105 loss is 0.007643852693339189
min_loss is:  0.00095575803 loss is 0.003918058161313335
min_loss is:  0.00094354013 loss is 0.003702365638067325
min_loss is:  0.00070240285 loss is 0.0038035587252428136
min_loss is:  0.00063157943 loss is 0.002596271806396544
min_loss is:  0.0006155587 loss is 0.00254555760572354
min_loss is:  0.00048916164 loss is 0.0024658939025054374
min_loss is:  0.00045985542 loss is 0.001495193496036033
min_loss is:  0.00044813217 loss is 0.0013994891196489334
min_loss is:  0.00039983654 loss is 0.001461029751226306
min_loss is:  0.00037377555 loss is 0.0013447258000572522
mi

In [18]:
model.eval()
y_pred1 = model.forward(x_train)
y_pred2 = model.forward(x_test)
y_pred = torch.vstack((y_pred1, y_pred2))

# y_true = torch.vstack((y_train, y_test))

y_pred = y_pred2
y_true = y_test

residual = y_pred - y_true
r2_values = torch.empty(y_pred.shape[0])
for i in range(y_pred.shape[0]):
    r2_values[i] = 1 - torch.sum(residual[i, :] ** 2) / torch.sum((y_true[i, :] - torch.mean(y_true[i, :])) ** 2)

average_r2 = r2_values.mean()

mae1 = mean_absolute_error(y_true, y_pred)
sdm1 = standard_deviation_error(y_true, y_pred)
mse = torch.mean((y_true - y_pred) ** 2)

print('mse:', mse)
print('mae:', mae1)
print('sdm:', sdm1)
print('r2:', average_r2)

mse: tensor(0.0007, device='cuda:0', grad_fn=<MeanBackward0>)
mae: tensor(0.0178, device='cuda:0', grad_fn=<MeanBackward0>)
sdm: tensor(0.0268, device='cuda:0', grad_fn=<StdBackward0>)
r2: tensor(0.9417, grad_fn=<MeanBackward0>)
